## Importing packages

In [1197]:
import json
import pandas as pd
import re
import time
import numpy as np
import random
import collections
import itertools
import math
from difflib import SequenceMatcher
import matplotlib.pyplot as plt

## Importing and cleaning the data

In [839]:
#import json data
with open('TVs-all-merged.json') as file:
    original_data=json.load(file)

new_data = {}
i = 1
for key in original_data.keys():
    for description in original_data[key]:
        new_data[i] = description
        i+=1
print(len(new_data.keys()))


1624


In [840]:
#Obtain bootstrapped data (about 60%)
bootData={}
numBootstraps=int(len(new_data.keys())*0.63) #63% of data for bootstrap
for i in range(1, numBootstraps+1):
    index=random.randint(1, len(new_data.keys()))
    #bootData[i]=new_data[index].items()
    bootData[i]=(new_data[index])

data=bootData

## Cleaning title and featuresMap input for hashing

In [841]:
#universal term for 'inch' and 'hz' and make everything lower case
def preprocess(text):
    CleaningDictionary = {'inch':['Inch', 'inches', '"', '-inch', ' inch', 'inch']
                    ,'hz':['Hertz', 'hertz', 'Hz', 'HZ', ' hz', '-hz', 'hz']}
    for normVal in CleaningDictionary.keys():
        possible=CleaningDictionary[normVal]
        for notNormVal in possible:
            text=text.replace(notNormVal, normVal)
    title=text.lower()
    title=re.sub("[^a-zA-Z0-9\s\.]","",title)
    return title

#preprocess all text in titles and featuresMap
for i in range(1, len(data.keys())+1):
    data[i]['title']=preprocess(data[i]['title'])
    for feature in data[i]['featuresMap']:
        data[i]['featuresMap'][feature]=preprocess(data[i]['featuresMap'][feature])


In [842]:
pattern='([a-zA-Z0-9]*(([0-9]+)|([0-9]+))[a-zA-Z0-9]*)'
shingles_title=[]
num_sig=512

#extract all model words from the title of the product
for i in range(1, len(data.keys())+1):
    for titleWords in re.findall(pattern, data[i]['title']):
        shingles_title.append(titleWords[0].strip()) #collection of all words
        
shingles=list(set(shingles_title))

#extract model words which contain letters-numbers-letters or numbers-letters-numbers and add them twice
modWordsPat='[a-zA-Z0-9][0-9]+[a-zA-Z]+[0-9]+[a-zA-Z0-9]|[a-zA-Z0-9]*[a-zA-Z]+[0-9]+[a-zA-Z]+[a-zA-Z0-9]+'
shingles_copy=shingles
for words in range(0, len(shingles)-1):
    if re.match(modWordsPat, shingles[words]):
        shingles_copy.append(shingles[words])
shingles=shingles_copy #modelIDs are added twice

#create a binary vector of model words for each product
for i in range(1, len(data.keys())+1):
    all_prods=[]
    bin_vecs=[]
    
    for titleWords in re.findall(pattern, data[i]['title']):
        all_prods.append(titleWords[0].strip())
    for word in shingles:
        if word in all_prods:
            bin_vecs.append(1)
        else:
            bin_vecs.append(0)
            
    data[i]['vector']=bin_vecs
    #data[i]['signature']=[int(len(bin_vecs)/2)] * (int(len(bin_vecs) /2)) 
    data[i]['signature']=[num_sig] * (num_sig)

## MinHash algorithm, creating signatures

In [1226]:
#check is a number is a prime number
def isPrime(n):
    for i in range(2,int(n**0.5)+1):
        if n%i==0:
            return False
    if n < int(len(bin_vecs) /2):
        return False
        
    return True

#construct the hash function for hashing the binary vectors to signatures
def HashFunction(a,b,p,r):
    return (a + b*r) % p 

primes = [i for i in range(0,len(shingles)) if isPrime(i)]

HashFrame = pd.DataFrame({'HashFunction':list(range(0,num_sig))
              ,'a': np.random.randint(0,len(shingles),size=num_sig)
              ,'b': np.random.randint(0,len(shingles),size=num_sig)
              ,'p': random.choices(primes,k=num_sig)})


integercounter = 0

#for the num_sig permutations of the binary vectors, add the number for first 1 in the vector as signature value
for i in range(1, len(data.keys())+1):
    for index,row in HashFrame.iterrows():
        x = 0
        for entry in data[i]['vector']:
            if entry == 1:
                data[i]['signature'][index] = min(data[i]['signature'][index],
                    HashFunction(int(row['a'])
                    ,int(row['b'])
                    ,int(row['p'])
                    ,x))
            x=x+1
    integercounter+=1

50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done
50 done


In [1227]:
signatures=pd.DataFrame(None)
for i in range(1, len(data.keys())+1):
    signatures=pd.concat([signatures, pd.DataFrame({i:data[i]['signature']})], axis=1)

In [1228]:
signatures

,1,2,3,4,5,6,7,8,9,10,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,171,36,22,134,35,166,97,98,22,25,...,36,30,11,36,32,102,98,425,47,36
1,76,21,7,115,220,115,3,80,20,115,...,241,216,12,34,115,33,40,118,28,44
2,8,35,8,8,163,8,8,8,8,8,...,17,268,8,93,8,8,8,253,8,59
3,120,78,43,35,88,34,7,54,50,146,...,36,132,35,132,54,25,22,456,7,258
4,305,32,21,3,97,264,21,21,9,49,...,42,122,67,156,21,10,21,113,42,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,1,1,8,74,250,13,76,71,55,6,...,69,110,74,12,37,223,74,114,1,18
508,50,81,6,6,131,60,6,6,132,7,...,29,47,3,59,6,57,6,512,29,371
509,57,18,133,126,0,1,14,77,82,12,...,15,12,34,18,133,23,76,45,15,18
510,139,7,126,254,82,35,34,28,14,14,...,7,105,13,7,28,232,28,198,27,7


## LSH procedure using the generated signatures

In [1229]:
#set values for b and r (consequently setting the value for threshold t)
n=len(signatures)
b=64
r=n/b

In [1230]:
#hash all products with the same signatures in a band to the same buckets
candidates=[]
desc=list(signatures.columns)
for band in range(0,b):
    buckets=collections.defaultdict(list)
    bandMatrix=signatures[(signatures.index>=band*r)&(signatures.index<band*r +r)]
    for entry in desc:
        buckets[tuple(bandMatrix[entry])].append(entry)
    candidate=[x for x in list(buckets.values()) if len(x)>1]
    for pair in candidate:
        if pair not in candidates:
            candidates.append(pair)
candidates.sort()

cands=candidates[:]

#products hashed to the same bucket at least once form candidate pairs
for m in candidates:
    for n in cands:
        if set(m).issubset(set(n)) and m!=n:
            cands.remove(m)
            break
candidates=cands

In [1231]:
len(candidates)

221

In [1232]:
#from the candidates, make all combinations of 2 products to construct the candidate pairs
comblist=[]
for x in candidates:
    if (len(x)==2)&(x not in comblist):
        comblist.append(x)
    else:
        for subset in itertools.combinations(x,2):
            if (list(subset) not in comblist):
                comblist.append(list(subset))

comblist.sort()
            

In [1233]:
len(comblist)

583

In [1234]:
candidates=comblist

## Computing the similarity and presenting estimated duplicates

In [1235]:
#preprocessing of the title and featureMap for comparison
def keyToString(key):
    key=re.sub(r'[^\w\s]','',key)
    string=key.lower()
    string=string.replace(" ", "")
    return string

#preprocessing of the featureMap for comparison
def createKeyQ(keys): #list of all keys as strings
    lstOfKeys=[]
    for key in keys:
        lstOfKeys.append(keyToString(key))
    return lstOfKeys

#calculate the similarity between two titles
similarities=[]
for pair in range(0, len(candidates)-1):
    init1=preprocess(data[candidates[pair][0]]['title'])
    init2=preprocess(data[candidates[pair][1]]['title'])
    title1=keyToString(init1)
    title2=keyToString(init2)
    sim=SequenceMatcher(None, title1, title2).ratio()
    similarities.append(sim)

#calculate the similarity between two featureMaps
for pair in range(0, len(candidates)-1): #for each pair in candidates
    #obtain list of strings for keys and values of pairs
    prod1=createKeyQ(data[candidates[pair][0]]['featuresMap'].keys())
    prod2=createKeyQ(data[candidates[pair][1]]['featuresMap'].keys())
    val1=createKeyQ(data[candidates[pair][0]]['featuresMap'].values())
    val2=createKeyQ(data[candidates[pair][1]]['featuresMap'].values())
    
    w=0
    sim=0
    avgSim=0
    newSim=0
    for i in range(0, len(prod1)):
        for j in range(0, len(prod2)):
            keySim=SequenceMatcher(None, prod1[i], prod2[j]).ratio()
            if keySim>=0.8: #if the keys of the featureMap match significantly, add the similarity of the values
                valueSim=SequenceMatcher(None, val1[i], val2[j]).ratio()
                weight=keySim
                sim=sim+(weight*valueSim)
                w=w+weight
    if w>0:
        avgSim=sim/w
    
    if avgSim != 0:
        newSim=(similarities[pair]+avgSim)/2
        similarities[pair]=newSim #similarity measure is average of the similarity between titles and featureMap


#propose all pairs with similarity>50% as duplicates
sameProducts=[i for i,e in enumerate(similarities) if e>=0.5]
    
vectorOfPairs=[]
for index, value in enumerate(sameProducts):
    vectorOfPairs.append(candidates[value])
    
modelIDs=[]
for i in range(1, len(data.keys())+1):
    #modelIDs[i]=data[i]['modelID']
    modelIDs.append(data[i]['modelID'])

#identify all true duplicate pairs
def listDuplicates(modelIDs):
    dups=collections.defaultdict(set)
    for index, product in enumerate(modelIDs):
        dups[product].add(index)
    duplicates=set()
    for match in dups.values():
        if len(match)>1:
            for pair in itertools.combinations(match,2):
                duplicates.add(pair)
    return duplicates

duplicates=listDuplicates(modelIDs)

TP=0
truePos=[]
trueMod=[]
#calculate the number of true positive duplicates
for i in range(0, len(vectorOfPairs)-1):
    if data[vectorOfPairs[i][0]]['modelID']==data[vectorOfPairs[i][1]]['modelID']:
        TP=TP+1 #number of true positives
        trueMod.append(data[vectorOfPairs[i][1]]['modelID'])
        truePos.append(vectorOfPairs[i])

FP=len(vectorOfPairs)-TP #number of false positives
FN=len(duplicates)-TP

#evaluation measures
pairCompl=TP/len(duplicates)
pairQual=TP/len(candidates)
precision=TP/(TP+FP)
recall=TP/(TP+FN)
F1=2*(precision*recall)/(precision+recall)
F1star=2*(pairQual*pairCompl)/(pairQual+pairCompl)

## Performance measures

In [1067]:
#FOR B=64, R=8, E>0.5
print('Pair completenes is: ' + str(pairCompl))
print('Pair quality is: ' + str(pairQual))
print('F1-measure is: ' + str(F1))
print('F1*-measure is: ' + str(F1star))
print('True positives: ' + str(TP))
print('False positives: ' + str(FP))
print('False negatives: ' + str(FN))
print('precision: ' + str(precision))
print('recall: ' + str(recall))

Pair completenes is: 0.8560606060606061
Pair quality is: 0.18880534670008353
F1-measure is: 0.3287272727272727
F1*-measure is: 0.30937713894592744
True positives: 452
False positives: 1770
False negatives: 76
precision: 0.20342034203420342
recall: 0.8560606060606061
